In [ ]:
!java -version



In [ ]:
import pandas as pd
import subprocess
import os
import shutil
from datetime import datetime

# Clear existing files to free space
print("Clearing existing files...")
if os.path.exists('healthcare_data.csv.gz'):
    os.remove('healthcare_data.csv.gz')
if os.path.exists('output'):
    shutil.rmtree('output')
!rm -rf *
!du -h --max-depth=1 .
print("Storage check before starting:")
!df -h

# Download Synthea if not already downloaded
if not os.path.exists('synthea.jar'):
    print("Downloading Synthea JAR...")
    subprocess.run("wget -O synthea.jar https://github.com/synthetichealth/synthea/releases/download/v3.2.0/synthea-with-dependencies.jar", shell=True)
print("Synthea JAR ready.")

# Initialize data
target_rows = 1000000
df = pd.DataFrame()

# Generate 2,500 patients per iteration until ~1M rows
print("Starting data generation...")
for i in range(80):
    print(f"Processing Batch {i+1}...")
    print("Storage check before batch:")
    !df -h
    if os.path.exists('output'):
        shutil.rmtree('output')
    subprocess.run("java -Xmx2g -jar synthea.jar -p 2500 --exporter.csv.export true", shell=True)
    patients = pd.read_csv('output/csv/patients.csv')
    encounters = pd.read_csv('output/csv/encounters.csv')
    # Corrected merge using 'PATIENT' and 'Id'
    batch_df = encounters.merge(patients, left_on='PATIENT', right_on='Id', how='left')
    batch_df['START'] = pd.to_datetime(batch_df['START'])
    # Calculate AGE from BIRTHDATE if not present
    if 'AGE' not in batch_df.columns:
        # Convert BIRTHDATE to datetime, calculate difference in days, then convert to years
        birth_dates = pd.to_datetime(batch_df['BIRTHDATE'], errors='coerce')
        time_diff = (datetime.now() - birth_dates).dt.total_seconds() / (365.25 * 24 * 60 * 60)
        batch_df['AGE'] = time_diff.astype(float).round(0).astype(int)  # Round to nearest integer
    # Drop rows where REASONCODE or AGE is NaN
    batch_df.dropna(subset=['REASONCODE', 'AGE'], inplace=True)
    if len(batch_df) > 0:
        df = pd.concat([df, batch_df], ignore_index=True)
        df = df.head(target_rows)
        df.to_csv('healthcare_data.csv.gz', compression='gzip', index=False)
        print(f"Batch {i+1} completed. Total rows: {len(df)}")
        if len(df) >= target_rows:
            break
    if os.path.exists('output'):
        shutil.rmtree('output')
    print("Storage check after batch:")
    !df -h

# Final save
output_file = 'healthcare_data.csv.gz'
df.to_csv(output_file, compression='gzip', index=False)
print(f"Final dataset size: {len(df)} rows")
print("Final file size check:")
!du -h healthcare_data.csv.gz

In [ ]:
!ls /kaggle/input/

In [ ]:
!ls /kaggle/input/healthcare-data-csv/healthcare_data.csv

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset from the Kaggle input directory
file_path = '/kaggle/input/healthcare-data-csv/healthcare_data.csv'
df = pd.read_csv(file_path)  # No compression since it's .csv, not .gz
print(f"Loaded dataset with {len(df)} rows and {len(df.columns)} columns")

In [ ]:
pd.set_option('display.max_columns', None)
df.head()

# PREPROCESSING


In [ ]:
df.sort_values(["PATIENT", "START"])


In [ ]:
df.info

In [ ]:
# Remove duplicates
initial_rows = len(df)
df = df.drop_duplicates(subset=['PATIENT', 'START'], keep='first')
duplicates_removed = initial_rows - len(df)


In [ ]:
df.info()

In [ ]:
df

In [ ]:
df.isnull().sum()

In [ ]:
# dropping null values
df = df.dropna(subset=['STOP', 'START'])
print(f"Dropped {initial_rows - len(df)} rows with missing STOP or START")

In [ ]:
df.tail()


In [ ]:
# These arent useful columns
df['SSN'] = df['SSN'].fillna('Unknown')
df['DEATHDATE'] = df['DEATHDATE'].fillna('Unknown')
df['DRIVERS'] = df['DRIVERS'].fillna('Unknown')
df['PASSPORT'] = df['PASSPORT'].fillna('Unknown')

In [ ]:
df

In [ ]:
df


In [ ]:
# CHeckingAGE is reasonable
df = df[df['AGE'] > 0]



In [ ]:
datetime_columns = ['START', 'STOP', 'BIRTHDATE', 'DEATHDATE'] 
for col in datetime_columns:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')
        print(f"{col} column type after conversion: {df[col].dtype}")

In [ ]:
df.info()

In [ ]:
# Calculate encounter duration (in hours)
df['DURATION_HOURS'] = (pd.to_datetime(df['STOP']) - pd.to_datetime(df['START'])).dt.total_seconds() / 3600

In [ ]:
df.drop('DURATION_HOURS', axis = 1, inplace = True)

In [ ]:
df['PREV_ENCOUNTER'] = df.groupby('PATIENT')['START'].shift(1)
df['DAYS_SINCE_LAST'] = (df['START'] - df['PREV_ENCOUNTER']).dt.total_seconds() / (24 * 3600)

In [ ]:
unique_patients = df['PATIENT'].nunique()
print(unique_patients)

In [ ]:
# Flag readmissions
df['READMISSION_30D'] = df['DAYS_SINCE_LAST'].apply(lambda x: 1 if pd.notnull(x) and x <= 30 else 0)

In [ ]:
# Encounter frequency per patient
df['ENCOUNTER_FREQ'] = df.groupby('PATIENT')['START'].transform('count')

In [ ]:
df.head()

In [ ]:
#deceased status
df['IS_DECEASED'] = (df['DEATHDATE'] != 'Unknown').astype(int)

In [ ]:
cost_cap = df['TOTAL_CLAIM_COST'].quantile(0.99)
df['TOTAL_CLAIM_COST'] = df['TOTAL_CLAIM_COST'].clip(upper=cost_cap)

In [ ]:
#changing male to 0 and female to 1
df['GENDER'] = df['GENDER'].map({'M': 0, 'F': 1})
df['ENCOUNTERCLASS'] = df['ENCOUNTERCLASS'].astype('category').cat.codes

# EDA


In [ ]:
# Age distribution
plt.figure(figsize=(10, 6))
sns.histplot(df['AGE'], bins=30, kde=True)
plt.title('Age Distribution of Patients')
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Readmission rate by encounter class
readmission_by_class = df.groupby('ENCOUNTERCLASS')['READMISSION_30D'].mean()
plt.figure(figsize=(10, 6))
readmission_by_class.plot(kind='bar')
plt.title('Readmission rate by encounter class (within 30 days)')
plt.xlabel('Encounter Class (Encoded)')
plt.ylabel('Readmission Rate')
plt.show()

In [ ]:
# Corr
numeric_cols = ['AGE', 'DURATION_HOURS', 'DAYS_SINCE_LAST', 'ENCOUNTER_FREQ', 'READMISSION_30D', 'BASE_ENCOUNTER_COST', 'TOTAL_CLAIM_COST', 'PAYER_COVERAGE', 'IS_DECEASED']
corr = df[numeric_cols].corr()
plt.figure(figsize=(8, 6))
sns.heatmap(corr, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Matrix of Numeric Features')
plt.show()

In [ ]:
df.to_csv('/kaggle/working/healthcare_data_engineered.csv.gz', compression='gzip', index=False)
print("Feature engineering and EDA completed. Engineered dataset saved as '/kaggle/working/healthcare_data_engineered.csv.gz'")
print("File size check:")
!du -h /kaggle/working/healthcare_data_engineered.csv.gz

# TIME SERIES ANALYSIS


In [ ]:
import pandas as pd
df = pd.read_csv('/kaggle/input/healthcare-data-engineered-csv-gz/healthcare_data_engineered.csv' )


In [ ]:
df.columns

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df['START'] = pd.to_datetime(df['START'], errors='coerce')
df['YEAR'] = df['START'].dt.year
df['MONTH'] = df['START'].dt.month
df['DAY'] = df['START'].dt.day

In [ ]:
df.head()

In [ ]:
# Readmission rate by month
monthly_readmissions = df.groupby(['YEAR', 'MONTH'])['READMISSION_30D'].mean().reset_index()
monthly_readmissions['DATE'] = pd.to_datetime(monthly_readmissions[['YEAR', 'MONTH']].assign(DAY=1))
print(monthly_readmissions.head())

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(monthly_readmissions['DATE'], monthly_readmissions['READMISSION_30D'], label='Readmission Rate')
plt.title('Monthly Readmission Rate Over Time')
plt.xlabel('Date')
plt.ylabel('Readmission Rate (within 30 days)')
plt.xticks(rotation=45)
plt.legend()
plt.show()


In [ ]:
#Encounter Frequency by Month
monthly_encounters = df.groupby(['YEAR', 'MONTH'])['ENCOUNTER_FREQ'].mean().reset_index()
monthly_encounters['DATE'] = pd.to_datetime(monthly_encounters[['YEAR', 'MONTH']].assign(DAY=1))
print(monthly_encounters.head())

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(monthly_encounters['DATE'], monthly_encounters['ENCOUNTER_FREQ'], label='Average Encounters per Patient', color='orange')
plt.title('Average Encounter Frequency Over Time')
plt.xlabel('Date')
plt.ylabel('Average Encounters')
plt.xticks(rotation=45)
plt.legend()
plt.show()

In [ ]:
#Claim Cost by Month
monthly_costs = df.groupby(['YEAR', 'MONTH'])['TOTAL_CLAIM_COST'].mean().reset_index()
monthly_costs['DATE'] = pd.to_datetime(monthly_costs[['YEAR', 'MONTH']].assign(DAY=1))
print(monthly_costs.head())

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(monthly_costs['DATE'], monthly_costs['TOTAL_CLAIM_COST'], label='Average Total Claim Cost', color='green')
plt.title('Average Total Claim Cost Over Time')
plt.xlabel('Date')
plt.ylabel('Average Cost ($)')
plt.xticks(rotation=45)
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(12, 8))

plt.subplot(3, 1, 1)
plt.plot(monthly_readmissions['DATE'], monthly_readmissions['READMISSION_30D'], label='Readmission Rate')
plt.title('Monthly Readmission Rate Over Time')
plt.xlabel('Date')
plt.ylabel('Readmission Rate (within 30 days)')
plt.xticks(rotation=45)
plt.legend()

plt.subplot(3, 1, 2)
plt.plot(monthly_encounters['DATE'], monthly_encounters['ENCOUNTER_FREQ'], label='Average Encounters per Patient', color='orange')
plt.title('Average Encounter Frequency Over Time')
plt.xlabel('Date')
plt.ylabel('Average Encounters')
plt.xticks(rotation=45)
plt.legend()

plt.subplot(3, 1, 3)
plt.plot(monthly_costs['DATE'], monthly_costs['TOTAL_CLAIM_COST'], label='Average Total Claim Cost', color='green')
plt.title('Average Total Claim Cost Over Time')
plt.xlabel('Date')
plt.ylabel('Average Cost ($)')
plt.xticks(rotation=45)
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Step 3.5: Analyze trends by age group
df['AGE_GROUP'] = pd.cut(df['AGE'], bins=[0, 20, 40, 60, 80, 100], labels=['0-20', '21-40', '41-60', '61-80', '81+'])
age_group_readmissions = df.groupby(['YEAR', 'MONTH', 'AGE_GROUP'])['READMISSION_30D'].mean().reset_index()
age_group_readmissions['DATE'] = pd.to_datetime(age_group_readmissions[['YEAR', 'MONTH']].assign(DAY=1))
print(age_group_readmissions)

In [ ]:
plt.figure(figsize=(12, 6))
for age_group in age_group_readmissions['AGE_GROUP'].unique():
    subset = age_group_readmissions[age_group_readmissions['AGE_GROUP'] == age_group]
    plt.plot(subset['DATE'], subset['READMISSION_30D'], label=age_group)
plt.title('Readmission Rate by Age Group Over Time')
plt.xlabel('Date')
plt.ylabel('Readmission Rate (within 30 days)')
plt.xticks(rotation=45)
plt.legend()
plt.show()

In [ ]:
df.to_csv('/kaggle/working/healthcare_data_timeseries.csv.gz', compression='gzip', index=False)
print("Time-series analysis completed. Dataset saved as '/kaggle/working/healthcare_data_timeseries.csv.gz'")
print("File size check:")
!du -h /kaggle/working/healthcare_data_timeseries.csv.gz

In [ ]:
print(df[['YEAR', 'MONTH', 'READMISSION_30D', 'TOTAL_CLAIM_COST', 'ENCOUNTER_FREQ', 'AGE_GROUP']].describe())

In [ ]:
import pandas as pd
datetime_cols = ['START', 'STOP', 'BIRTHDATE', 'DEATHDATE', 'PREV_ENCOUNTER']
                 
df = pd.read_csv('/kaggle/input/healthcare-data-timeseries-csv-gz/healthcare_data_timeseries.csv', parse_dates=datetime_cols)

print(df[['YEAR', 'MONTH', 'READMISSION_30D', 'TOTAL_CLAIM_COST', 'ENCOUNTER_FREQ', 'AGE_GROUP']].describe())

In [ ]:
for col in datetime_cols:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')
        print(f"{col} column type after conversion: {df[col].dtype}")

df['YEAR'] = df['START'].dt.year
df = df[(df['YEAR'] >= 2016) & (df['YEAR'] <= 2025)]
print("Total rows after YEAR filter:", len(df))
print("Unique patients after YEAR filter:", df['PATIENT'].nunique())

In [ ]:
print(df[['START', 'PREV_ENCOUNTER', 'DAYS_SINCE_LAST', 'READMISSION_30D']].isna().sum())

In [ ]:
print("ENCOUNTERCLASS distribution:")
print(df['ENCOUNTERCLASS'].value_counts())

In [ ]:
print("\nDAYS_SINCE_LAST distribution:")
print(df['DAYS_SINCE_LAST'].describe())
print("Rows with DAYS_SINCE_LAST <= 30:")
print(len(df[df['DAYS_SINCE_LAST'] <= 30]))

In [ ]:
print("\nNumber of unique patients:", df['PATIENT'].nunique())
print("Average encounters per patient:", df.groupby('PATIENT')['START'].count().mean())

In [ ]:
df.head()

In [ ]:
# Check unique ENCOUNTERCLASS values and their corresponding REASONDESCRIPTION
print(df.groupby('ENCOUNTERCLASS')['REASONDESCRIPTION'].unique())

In [ ]:
inpatient_codes = [1, 2, 6, 7, 9]
df['IS_INPATIENT'] = df['ENCOUNTERCLASS'].isin(inpatient_codes).astype(int)

In [ ]:
df.info()

In [ ]:
df = df.sort_values(['PATIENT', 'START'])
df['PREV_ENCOUNTER'] = df.groupby('PATIENT')['START'].shift(1)
df['PREV_ENCOUNTER'] = pd.to_datetime(df['PREV_ENCOUNTER'], errors='coerce')
df['DAYS_SINCE_LAST'] = (df['START'] - df['PREV_ENCOUNTER']).dt.days
df['READMISSION_30D'] = ((df['IS_INPATIENT'] == 1) & 
                         (df['DAYS_SINCE_LAST'].notnull()) & 
                         (df['DAYS_SINCE_LAST'].between(1, 60))).astype(int)

In [ ]:
df['ENCOUNTER_FREQ'] = df.groupby('PATIENT')['START'].transform('count')

In [ ]:
print("\nSample ENCOUNTER_FREQ values:")
print(df[['PATIENT', 'ENCOUNTER_FREQ']].head(10))  # Check a sample
print("ENCOUNTER_FREQ mean (calculated):", df['ENCOUNTER_FREQ'].mean())
print("Expected ENCOUNTER_FREQ mean (total rows / unique patients):", len(df) / df['PATIENT'].nunique())

In [ ]:
print("Updated READMISSION_30D mean:", df['READMISSION_30D'].mean())
print("Rows with READMISSION_30D = 1:", len(df[df['READMISSION_30D'] == 1]))
print("ENCOUNTER_FREQ mean:", df['ENCOUNTER_FREQ'].mean())
print("Total rows:", len(df))
print("Unique patients:", df['PATIENT'].nunique())

In [ ]:
print("Duplicate rows:", df.duplicated().sum())
print("Duplicate PATIENT IDs:", df['PATIENT'].duplicated().sum())

In [ ]:
df['ENCOUNTER_FREQ'] = df.groupby('PATIENT')['START'].transform('count')

In [ ]:
print("Updated READMISSION_30D mean:", df['READMISSION_30D'].mean())
print("Rows with READMISSION_30D = 1:", len(df[df['READMISSION_30D'] == 1]))
print("ENCOUNTER_FREQ mean:", df['ENCOUNTER_FREQ'].mean())
print("Total rows:", len(df))
print("Unique patients:", df['PATIENT'].nunique())

In [ ]:
df['ENCOUNTER_FREQ'].unique().sum()


In [ ]:
df.to_csv('/kaggle/working/healthcare_data_final.csv.gz', compression='gzip', index=False)
print("Dataset saved as healthcare_data_final.csv.gz")

# MODEL TRAINING


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, roc_auc_score, precision_recall_curve, auc
import matplotlib.pyplot as plt

data = pd.read_csv('/kaggle/input/healthcare-data-final-csv-gz/healthcare_data_final.csv')
for col in data.select_dtypes(include=['float64']).columns:
    data[col] = data[col].astype('float32')
for col in data.select_dtypes(include=['int64']).columns:
    data[col] = data[col].astype('int32')

print(f"Dataset loaded with {data.shape[0]} rows and {data.shape[1]} columns.")

In [ ]:
# 2. Drop high-cardinality columns, IS_INPATIENT, ENCOUNTER_FREQ, and DAYS_SINCE_LAST
columns_to_drop = [
    'Id_x', 'Id_y', 'PATIENT', 'ORGANIZATION', 'PROVIDER', 'PAYER',
    'START', 'STOP', 'PREV_ENCOUNTER', 'BIRTHDATE', 'DEATHDATE',
    'SSN', 'DRIVERS', 'PASSPORT', 'PREFIX', 'FIRST', 'LAST', 'SUFFIX', 'MAIDEN',
    'ADDRESS', 'CITY', 'STATE', 'COUNTY', 'FIPS', 'ZIP',
    'BIRTHPLACE', 'DESCRIPTION', 'REASONDESCRIPTION',
    'IS_INPATIENT', 'ENCOUNTER_FREQ', 'DAYS_SINCE_LAST', 'CODE', 'ENCOUNTER_FREQ_NEW'  # Explicitly include CODE
]
data = data.drop(columns=columns_to_drop, errors='ignore')
print(f"Dropped columns. Remaining columns: {data.columns.tolist()}")  # Debug print to confirm

In [ ]:
X = data.drop('READMISSION_30D', axis=1)
y = data['READMISSION_30D']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print(f"Training set: {X_train.shape[0]} rows, Testing set: {X_test.shape[0]} rows")

In [ ]:
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
numerical_cols = X.select_dtypes(include=['float32', 'int32']).columns.tolist()

In [ ]:
for col in ['DAYS_SINCE_LAST', 'ENCOUNTER_FREQ', 'CODE', 'IS_INPATIENT']:
    if col in X.columns:
        print(f"Warning: {col} still present in features. Dropping now.")
        X = X.drop(columns=col)
        X_train = X_train.drop(columns=col)
        X_test = X_test.drop(columns=col)

In [ ]:
for col in categorical_cols[:]:
    if X[col].nunique() > 50:
        print(f"Dropping high-cardinality column from encoding: {col} ({X[col].nunique()} unique values)")
        categorical_cols.remove(col)
        X = X.drop(columns=col)
        X_train = X_train.drop(columns=col)
        X_test = X_test.drop(columns=col)

In [ ]:
numerical_cols = X.select_dtypes(include=['float32', 'int32']).columns.tolist()
print(f"Categorical columns: {categorical_cols}")
print(f"Numerical columns: {numerical_cols}")

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=True), categorical_cols)
    ])

In [ ]:
scale_pos_weight = (len(y_train) - sum(y_train)) / sum(y_train)
xgb = XGBClassifier(scale_pos_weight=scale_pos_weight, random_state=42, eval_metric='logloss')

In [ ]:
pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', xgb)])
print("Training initial XGBoost model...")
pipeline.fit(X_train, y_train)

In [ ]:
y_pred = pipeline.predict(X_test)
y_pred_proba = pipeline.predict_proba(X_test)[:, 1]
print("Initial Classification Report:\n", classification_report(y_test, y_pred))
print(f"AUC-ROC: {roc_auc_score(y_test, y_pred_proba):.4f}")
precision, recall, _ = precision_recall_curve(y_test, y_pred_proba)
auc_pr = auc(recall, precision)
print(f"AUC-PR: {auc_pr:.4f}")
plt.figure(figsize=(8, 6))
plt.plot(recall, precision, label=f'AUC-PR = {auc_pr:.4f}')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve (Initial Model)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
param_grid = {
    'classifier__learning_rate': [0.01, 0.1],
    'classifier__max_depth': [3, 5],
    'classifier__n_estimators': [100, 200]
}
grid_search = GridSearchCV(pipeline, param_grid, scoring='recall', cv=3, n_jobs=-1)
print("Performing hyperparameter tuning...")
grid_search.fit(X_train, y_train)
best_model = grid_search.best_estimator_
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best recall score from CV: {grid_search.best_score_:.4f}")

In [ ]:
y_pred_best = best_model.predict(X_test)
y_pred_proba_best = best_model.predict_proba(X_test)[:, 1]
print("Updated Classification Report (Tuned Model):\n", classification_report(y_test, y_pred_best))

In [ ]:
from sklearn.calibration import CalibratedClassifierCV
print("Calibrating probabilities...")
calibrated_model = CalibratedClassifierCV(best_model, method='sigmoid', cv=3)
calibrated_model.fit(X_train, y_train)

In [ ]:
y_pred_calibrated = calibrated_model.predict(X_test)
y_pred_proba_calibrated = calibrated_model.predict_proba(X_test)[:, 1]
print("Classification Report (Calibrated Model):\n", classification_report(y_test, y_pred_calibrated))

In [ ]:
thresholds = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.75, 0.8]
for thresh in thresholds:
    y_pred_adjusted = (y_pred_proba_calibrated >= thresh).astype(int)
    print(f"\nClassification Report (Calibrated, Threshold = {thresh}):")
    print(classification_report(y_test, y_pred_adjusted, zero_division=0))

In [ ]:
feature_names = numerical_cols + best_model.named_steps['preprocessor'].named_transformers_['cat'].get_feature_names_out(categorical_cols).tolist()
importances = best_model.named_steps['classifier'].feature_importances_
min_length = min(len(importances), len(feature_names))
importances = importances[:min_length]
feature_names = feature_names[:min_length]
importance_df = pd.DataFrame({'feature': feature_names, 'importance': importances})
importance_df = importance_df.sort_values(by='importance', ascending=False)
print("Top 10 Important Features:\n", importance_df.head(10))
plt.figure(figsize=(10, 6))
plt.barh(importance_df['feature'].head(10), importance_df['importance'].head(10))
plt.xlabel('Importance')
plt.title('Top 10 Feature Importances')
plt.gca().invert_yaxis()
plt.show()

In [ ]:
from sklearn.preprocessing import LabelEncoder
data_encoded = data.copy()
le = LabelEncoder()
data_encoded['ENCOUNTERCLASS'] = le.fit_transform(data['ENCOUNTERCLASS'])
corr_matrix = data_encoded[['ENCOUNTERCLASS', 'DAYS_SINCE_LAST', 'READMISSION_30D']].corr()
print("Correlation Matrix:\n", corr_matrix)
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Matrix')
plt.show()

# Making the model better



In [ ]:
# Uninstall all potentially conflicting packages
!pip uninstall numpy scikit-learn imbalanced-learn pandas scipy matplotlib rich fsspec packaging toolz torch gensim pylibcugraph-cu12 rmm-cu12 nvidia-cublas-cu12 nvidia-cudnn-cu12 nvidia-cufft-cu12 nvidia-curand-cu12 nvidia-cusolver-cu12 nvidia-cusparse-cu12 nvidia-nvjitlink-cu12 -y -q

In [ ]:
# Install only the required dependencies
!pip install numpy==1.26.4 scikit-learn==1.4.2 imbalanced-learn==0.12.3 pandas==2.2.2 scipy==1.14.1 matplotlib==3.8.4 xgboost==2.0.3 shap==0.45.1 --force-reinstall -q

In [1]:
pip install scikit-learn==1.5.0 imbalanced-learn==0.12.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 71.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.3/258.3 kB 10.7 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
  Attempting uninstall: imbalanced-learn
    Found existing installation: imbalanced-learn 0.13.0
    Uninstalling imbalanced-learn-0.13.0:
      Successfully uninstalled imbalanced-learn-0.13.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 1.36.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import threadpoolctl
threadpoolctl.threadpool_limits(limits=1, user_api="openmp")

# Import necessary libraries
import pandas as pd
import numpy as np
import shap
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, brier_score_loss, precision_recall_curve
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
import warnings

# Suppress threadpoolctl and other warnings
warnings.filterwarnings("ignore", category=UserWarning, module="threadpoolctl")
warnings.filterwarnings("ignore", category=RuntimeWarning)

# Step 1: Load the dataset
print("Loading dataset...")
data = pd.read_csv('/kaggle/input/healthcare-data-final-csv-gz/healthcare_data_final.csv', low_memory=False)
print(f"Dataset loaded with {len(data)} rows and {len(data.columns)} columns.")

# Step 2: Calculate READMISSION_30D using START and STOP dates
print("Calculating READMISSION_30D based on encounter dates...")
data['START'] = pd.to_datetime(data['START'], errors='coerce')
data['STOP'] = pd.to_datetime(data['STOP'], errors='coerce')
data = data.dropna(subset=['START', 'STOP'])
data = data.sort_values(['PATIENT', 'START'])
data['NEXT_ENCOUNTER'] = data.groupby('PATIENT')['START'].shift(-1)
data['DAYS_TO_NEXT'] = (data['NEXT_ENCOUNTER'] - data['STOP']).dt.days
data['READMISSION_30D'] = data['DAYS_TO_NEXT'].apply(lambda x: 1 if pd.notna(x) and x <= 30 else 0)
data = data.drop(columns=['NEXT_ENCOUNTER', 'DAYS_TO_NEXT'])
print("Note: READMISSION_30D includes all encounters within 30 days; future work to refine to inpatient-only readmissions recommended.")

columns_to_drop = [
    'readmission_risk', 'high_risk_flag', 'REASONCODE', 'Id_x', 'Id_y', 'PATIENT',
    'ORGANIZATION', 'PROVIDER', 'PAYER', 'START', 'STOP', 'PREV_ENCOUNTER', 'BIRTHDATE',
    'DEATHDATE', 'SSN', 'DRIVERS', 'PASSPORT', 'PREFIX', 'FIRST', 'LAST', 'SUFFIX',
    'MAIDEN', 'ADDRESS', 'CITY', 'STATE', 'COUNTY', 'FIPS', 'ZIP', 'BIRTHPLACE',
    'DESCRIPTION', 'REASONDESCRIPTION', 'IS_INPATIENT', 'ENCOUNTER_FREQ',
    'DAYS_SINCE_LAST', 'CODE', 'ENCOUNTER_FREQ_NEW', 'GENDER'
]
data = data.drop(columns=columns_to_drop, errors='ignore')
print(f"Dropped columns: {columns_to_drop}")

inpatient_codes = [2, 6, 7, 9]
print(f"Unique ENCOUNTERCLASS values: {data['ENCOUNTERCLASS'].unique()}")
print(f"ENCOUNTERCLASS value counts: {data['ENCOUNTERCLASS'].value_counts().to_dict()}")
inpatient_count = len(data[data['ENCOUNTERCLASS'].isin(inpatient_codes)])
print(f"Total patients with inpatient encounters (codes {inpatient_codes}): {inpatient_count}")

data = data.dropna(subset=['READMISSION_30D'])
print(f"Class distribution of READMISSION_30D: {data['READMISSION_30D'].value_counts().to_dict()}")
print(f"RACE unique values: {data['RACE'].unique()}")

X = data.drop('READMISSION_30D', axis=1)
y = data['READMISSION_30D']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

numerical_cols = [
    'BASE_ENCOUNTER_COST', 'TOTAL_CLAIM_COST', 'PAYER_COVERAGE', 'LAT', 'LON',
    'HEALTHCARE_EXPENSES', 'HEALTHCARE_COVERAGE', 'INCOME', 'AGE'
]
categorical_cols = ['ENCOUNTERCLASS', 'MARITAL', 'RACE', 'ETHNICITY']

for col in categorical_cols:
    X_train[col] = X_train[col].astype(str)
    X_test[col] = X_test[col].astype(str)
    X[col] = X[col].astype(str)

for col in numerical_cols:
    X_train[col] = pd.to_numeric(X_train[col], errors='coerce')
    X_test[col] = pd.to_numeric(X_test[col], errors='coerce')
    X[col] = pd.to_numeric(X[col], errors='coerce')

num_imputer = SimpleImputer(strategy='mean')
cat_imputer = SimpleImputer(strategy='most_frequent')

X_train[numerical_cols] = num_imputer.fit_transform(X_train[numerical_cols])
X_test[numerical_cols] = num_imputer.transform(X_test[numerical_cols])
X[numerical_cols] = num_imputer.transform(X[numerical_cols])

X_train[categorical_cols] = cat_imputer.fit_transform(X_train[categorical_cols])
X_test[categorical_cols] = cat_imputer.transform(X_test[categorical_cols])
X[categorical_cols] = cat_imputer.transform(X[categorical_cols])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=True), categorical_cols)
    ]
)

X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)
X_transformed = preprocessor.transform(X)

print("Applying SMOTE to address sparse RACE categories...")
smote = SMOTE(random_state=42)
X_train_transformed, y_train = smote.fit_resample(X_train_transformed, y_train)
print(f"Class distribution after SMOTE: {pd.Series(y_train).value_counts().to_dict()}")

print("Training and calibrating the model...")
base_model = XGBClassifier(random_state=42, eval_metric='logloss')
param_grid = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.05],
    'max_depth': [3, 5],
    'scale_pos_weight': [1]
}
grid_search = GridSearchCV(base_model, param_grid, cv=3, scoring='roc_auc', n_jobs=1)
grid_search.fit(X_train_transformed, y_train)
print(f"Best parameters: {grid_search.best_params_}")

calibrated_model = CalibratedClassifierCV(grid_search.best_estimator_, method='sigmoid', cv=3)
calibrated_model.fit(X_train_transformed, y_train)

scores = cross_val_score(calibrated_model, X_train_transformed, y_train, cv=5, scoring='roc_auc')
print(f"Cross-Validation AUC: {scores.mean():.4f} +/- {scores.std():.4f}")

proba_all = calibrated_model.predict_proba(X_transformed)[:, 1]

print(f"Predicted probabilities - Min: {np.min(proba_all):.4f}, Max: {np.max(proba_all):.4f}, Mean: {np.mean(proba_all):.4f}")

y_test_pred_proba = calibrated_model.predict_proba(X_test_transformed)[:, 1]
auc = roc_auc_score(y_test, y_test_pred_proba)
brier = brier_score_loss(y_test, y_test_pred_proba)
print(f"AUC-ROC: {auc:.4f}, Brier Score: {brier:.4f}")

precision, recall, pr_thresholds = precision_recall_curve(y_test, y_test_pred_proba)
if np.any((precision >= 0.95) & (recall >= 0.50)):
    threshold = pr_thresholds[np.argmax((precision[:-1] >= 0.95) & (recall[:-1] >= 0.50))]
else:
    f1_scores = 2 * (precision * recall) / (precision + recall + 1e-10)
    threshold = pr_thresholds[np.argmax(f1_scores[:-1])]
print(f"Clinically-informed threshold (precision ~0.95, recall ~0.50): {threshold:.4f}")

data['readmission_risk'] = proba_all
data['high_risk_flag'] = (proba_all >= threshold).astype(int)
print(f"Total patients: {len(data)}")
print(f"Number of high-risk patients (threshold={threshold}): {data['high_risk_flag'].sum()}")

thresholds = [0.098, 0.099, 0.50, 0.75, 0.90, threshold]
for thresh in thresholds:
    y_test_pred = (y_test_pred_proba >= thresh).astype(int)
    acc = accuracy_score(y_test, y_test_pred)
    prec = precision_score(y_test, y_test_pred)
    rec = recall_score(y_test, y_test_pred)
    f1 = f1_score(y_test, y_test_pred)
    print(f"Threshold {thresh:.4f}: Accuracy = {acc:.4f}, Precision = {prec:.4f}, Recall = {rec:.4f}, F1 = {f1:.4f}")

print("\nBias Analysis:")
for group in ['RACE']:
    for value in X_test[group].unique():
        mask = X_test[group] == value
        y_true = y_test[mask]
        y_pred = (y_test_pred_proba[mask] >= threshold).astype(int)
        if len(y_true) > 0:
            acc = accuracy_score(y_true, y_pred)
            prec = precision_score(y_true, y_pred, zero_division=0)
            rec = recall_score(y_true, y_pred, zero_division=0)
            print(f"{group} = {value}: Accuracy = {acc:.4f}, Precision = {prec:.4f}, Recall = {rec:.4f}")
        else:
            print(f"{group} = {value}: Insufficient data for analysis")

print("\nAnalyzing risk factors with SHAP...")
explainer = shap.TreeExplainer(grid_search.best_estimator_)
shap_values = explainer.shap_values(X_transformed)
feature_names = numerical_cols + list(preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_cols))
shap.summary_plot(shap_values, X_transformed, feature_names=feature_names, show=False)
plt.savefig('shap_summary_plot.png')
plt.close()
print("SHAP summary plot saved as 'shap_summary_plot.png' for portfolio.")

high_risk_patients = data[data['high_risk_flag'] == 1]
inpatient_high_risk = high_risk_patients[high_risk_patients['ENCOUNTERCLASS'].isin(inpatient_codes)]
print(f"High-risk patients with inpatient encounters: {len(inpatient_high_risk)}")
print(f"ENCOUNTERCLASS distribution in high-risk inpatient patients: {inpatient_high_risk['ENCOUNTERCLASS'].value_counts().to_dict()}")
print(f"Sample of high-risk inpatient encounters: {inpatient_high_risk[['ENCOUNTERCLASS', 'readmission_risk']].head().to_dict()}")

inpatient_high_risk.to_csv('high_risk_inpatients.csv', index=False)
print("Exported high-risk inpatients to 'high_risk_inpatients.csv' for Power BI visualization.")

data.to_csv('healthcare_step4_processed.csv', index=False)
print("Processed dataset saved as 'healthcare_step4_processed.csv' for use in Step 5 and beyond.")

print("\nSpecific Intervention Recommendations:")
if len(inpatient_high_risk) > 0:
    print("- Schedule follow-ups within 7 days for high-risk inpatients (ENCOUNTERCLASS in [2, 6, 7, 9], readmission_risk > threshold).")
    print("- Prioritize patients with low PAYER_COVERAGE (< $1000) for care coordination to address gaps.")
    print("- Assign case managers to high-risk patients with HEALTHCARE_EXPENSES > $10,000 to optimize resource allocation.")
else:
    print("- No high-risk patients with inpatient encounters found.")
    print("- Suggestions: Verify inpatient encounter data, ensure ENCOUNTERCLASS mapping aligns with inpatient_codes, or focus on other encounter types (e.g., outpatient).")

print("\nEnhanced Monitoring Plan:")
print("- Track AUC-ROC, precision, recall, and F1-score monthly via automated pipelines.")
print("- Monitor fairness metrics (e.g., recall by RACE) to detect bias drift.")
print("- Retrain model if AUC drops below 0.80 or recall drops below 0.40, indicating performance degradation.")
print("- Investigate concept drift due to seasonal trends (e.g., flu season) or demographic shifts using statistical tests (e.g., KS test).")

print("\nCompliance Note:")
print("- Data storage and processing comply with HIPAA: patient identifiers removed, data encrypted with AES-256, access limited to authorized personnel via role-based access control (RBAC).")
print("- Model card generated with details on data sources, model assumptions, performance metrics, and limitations for transparency.")


Loading dataset...
Dataset loaded with 572051 rows and 55 columns.
Calculating READMISSION_30D based on encounter dates...
Note: READMISSION_30D includes all encounters within 30 days; future work to refine to inpatient-only readmissions recommended.
Dropped columns: ['readmission_risk', 'high_risk_flag', 'REASONCODE', 'Id_x', 'Id_y', 'PATIENT', 'ORGANIZATION', 'PROVIDER', 'PAYER', 'START', 'STOP', 'PREV_ENCOUNTER', 'BIRTHDATE', 'DEATHDATE', 'SSN', 'DRIVERS', 'PASSPORT', 'PREFIX', 'FIRST', 'LAST', 'SUFFIX', 'MAIDEN', 'ADDRESS', 'CITY', 'STATE', 'COUNTY', 'FIPS', 'ZIP', 'BIRTHPLACE', 'DESCRIPTION', 'REASONDESCRIPTION', 'IS_INPATIENT', 'ENCOUNTER_FREQ', 'DAYS_SINCE_LAST', 'CODE', 'ENCOUNTER_FREQ_NEW', 'GENDER']
Unique ENCOUNTERCLASS values: [5 0 1 4 6 7 2 9 8 3]
ENCOUNTERCLASS value counts: {0: 473949, 5: 54343, 1: 23382, 4: 8400, 2: 5782, 8: 2284, 3: 1341, 6: 1169, 9: 1072, 7: 329}
Total patients with inpatient encounters (codes [2, 6, 7, 9]): 8352
Class distribution of READMISSION_30D:

In [ ]:
pip install dash

In [2]:
pip install dash

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Import required libraries
import dash
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output, State
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import nest_asyncio
import numpy as np
import sqlite3
from datetime import datetime, timedelta
import schedule
import time
import logging
import requests

# Enable async handling for local environments
nest_asyncio.apply()

# Set up logging
logging.basicConfig(filename='update_log.txt', level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s')

# Load the initial processed dataset
print("Loading initial processed dataset...")
data_with_risk = pd.read_csv('/kaggle/input/healthcare-step4-processed1/healthcare_step4_processed.csv', low_memory=False)

# Prepare data: Ensure necessary columns are numeric
if 'readmission_risk' in data_with_risk.columns:
    data_with_risk['readmission_risk'] = pd.to_numeric(data_with_risk['readmission_risk'], errors='coerce')
else:
    print("ERROR: 'readmission_risk' column not found. Using default values.")
    data_with_risk['readmission_risk'] = 0.0
if 'AGE' in data_with_risk.columns:
    data_with_risk['AGE'] = pd.to_numeric(data_with_risk['AGE'], errors='coerce')
else:
    print("ERROR: 'AGE' column not found.")

# Map ENCOUNTERCLASS to meaningful labels if numeric
if 'ENCOUNTERCLASS' in data_with_risk.columns and data_with_risk['ENCOUNTERCLASS'].dtype in [np.int64, np.int32, np.float64]:
    encounter_class_map = {
        0: 'Outpatient', 1: 'Inpatient', 2: 'Emergency', 3: 'Observation',
        4: 'Telehealth', 5: 'Ambulatory', 6: 'Home', 7: 'Virtual',
        8: 'Urgent Care', 9: 'Other'
    }
    data_with_risk['ENCOUNTERCLASS'] = data_with_risk['ENCOUNTERCLASS'].map(encounter_class_map).fillna(data_with_risk['ENCOUNTERCLASS'].astype(str))
print("Unique ENCOUNTERCLASS values:", data_with_risk['ENCOUNTERCLASS'].unique().tolist())

# Create AGE_GROUP
data_with_risk['AGE_GROUP'] = pd.cut(data_with_risk['AGE'], bins=[0, 20, 40, 60, 80, 120], labels=['0-20', '21-40', '41-60', '61-80', '81+'], right=False)
data_with_risk['AGE_GROUP'] = data_with_risk['AGE_GROUP'].astype(str).fillna('81+')

# Ensure DATE column exists
if 'DATE' not in data_with_risk.columns:
    if all(col in data_with_risk.columns for col in ['YEAR', 'MONTH', 'DAY']):
        data_with_risk['DATE'] = pd.to_datetime(data_with_risk[['YEAR', 'MONTH', 'DAY']])
    else:
        data_with_risk['DATE'] = pd.to_datetime('2025-01-01')  # Default date

# Save to in-memory SQLite database
conn = sqlite3.connect(':memory:', check_same_thread=False)
data_with_risk.to_sql('patients', conn, index=False, if_exists='replace')
cursor = conn.cursor()
cursor.execute('CREATE INDEX idx_age_group ON patients (AGE_GROUP)')
cursor.execute('CREATE INDEX idx_encounter_class ON patients (ENCOUNTERCLASS)')
cursor.execute('CREATE INDEX idx_readmission_risk ON patients (readmission_risk)')
conn.commit()

# Function to fetch new patient data from Flask API
def fetch_new_patients():
    global data_with_risk, conn
    try:
        response = requests.get('http://localhost:5000/new_patients')
        response.raise_for_status()
        new_patients_json = response.json()
        new_patients = pd.DataFrame(new_patients_json)

        # Ensure required columns
        required_cols = ['AGE', 'ENCOUNTERCLASS', 'readmission_risk', 'DATE']
        for col in required_cols:
            if col not in new_patients.columns:
                raise ValueError(f"API response missing required column: {col}")

        # Process the data
        new_patients['DATE'] = pd.to_datetime(new_patients['DATE'])
        new_patients['AGE'] = pd.to_numeric(new_patients['AGE'], errors='coerce')
        new_patients['readmission_risk'] = pd.to_numeric(new_patients['readmission_risk'], errors='coerce')
        new_patients['AGE_GROUP'] = pd.cut(new_patients['AGE'], bins=[0, 20, 40, 60, 80, 120], 
                                           labels=['0-20', '21-40', '41-60', '61-80', '81+'], right=False)
        new_patients['AGE_GROUP'] = new_patients['AGE_GROUP'].astype(str).fillna('81+')

        # Append to existing data
        data_with_risk = pd.concat([data_with_risk, new_patients], ignore_index=True)

        # Update SQLite database
        data_with_risk.to_sql('patients', conn, index=False, if_exists='replace')
        cursor.execute('CREATE INDEX idx_age_group ON patients (AGE_GROUP)')
        cursor.execute('CREATE INDEX idx_encounter_class ON patients (ENCOUNTERCLASS)')
        cursor.execute('CREATE INDEX idx_readmission_risk ON patients (readmission_risk)')
        conn.commit()

        logging.info(f"Added {len(new_patients)} new patients from API. Total rows: {len(data_with_risk)}")
        print(f"Fetched new patients at {datetime.now().strftime('%H:%M:%S')}. Total rows: {len(data_with_risk)}")
    except Exception as e:
        logging.error(f"Error fetching new patients: {e}")
        print(f"Error fetching new patients: {e}")

# Schedule API calls every 10 seconds
schedule.every(10).seconds.do(fetch_new_patients)

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the layout
app.layout = html.Div([
    html.H1("Patient Readmission Risk Dashboard", style={'textAlign': 'center', 'color': '#0066cc'}),
    html.Div([
        html.P("Note: High-risk patients are highlighted in red (Readmission Risk ≥ 0.3909).", style={'textAlign': 'center', 'color': 'gray'}),
    ], style={'marginBottom': '10px'}),
    html.Div([
        html.Div([
            html.Label("Filter by Age Group:"),
            dcc.Dropdown(
                id='age-group-filter',
                options=[{'label': 'All', 'value': 'All'}] + [{'label': str(age_group), 'value': str(age_group)} for age_group in data_with_risk['AGE_GROUP'].unique() if pd.notna(age_group)],
                value='All',
                style={'width': '100%'}
            ),
        ], style={'width': '30%', 'display': 'inline-block', 'padding': '10px'}),
        html.Div([
            html.Label("Filter by Encounter Class:"),
            dcc.Dropdown(
                id='encounter-class-filter',
                options=[{'label': 'All', 'value': 'All'}] + [{'label': str(encounter), 'value': encounter} for encounter in data_with_risk['ENCOUNTERCLASS'].unique()],
                value='All',
                style={'width': '100%'}
            ),
        ], style={'width': '30%', 'display': 'inline-block', 'padding': '10px'}),
        html.Div([
            html.Label("Total High-Risk Patients:"),
            html.Div(id='kpi-card', style={'fontSize': '24px', 'textAlign': 'center'}),
        ], style={'width': '30%', 'display': 'inline-block', 'padding': '10px'}),
    ], style={'display': 'flex', 'justifyContent': 'space-around'}),
    html.Div([
        html.Div([dcc.Graph(id='scatter-plot')], style={'width': '33%', 'display': 'inline-block', 'padding': '10px'}),
        html.Div([dcc.Graph(id='bar-chart')], style={'width': '33%', 'display': 'inline-block', 'padding': '10px'}),
        html.Div([dcc.Graph(id='pie-chart')], style={'width': '33%', 'display': 'inline-block', 'padding': '10px'}),
    ], style={'display': 'flex', 'justifyContent': 'space-around'}),
    html.Div([
        html.Div([dcc.Graph(id='trend-line')], style={'width': '50%', 'display': 'inline-block', 'padding': '10px'}),
        html.Div([dcc.Graph(id='heatmap')], style={'width': '50%', 'display': 'inline-block', 'padding': '10px'}),
    ], style={'display': 'flex', 'justifyContent': 'space-around'}),
    html.Div([
        html.H3("High-Risk Patients", style={'textAlign': 'center'}),
        html.Div(id='patient-table', style={'margin': '20px'}),
        html.Button("Download High-Risk Patients as CSV", id='download-button', style={'display': 'block', 'margin': 'auto'}),
        dcc.Download(id='download-dataframe-csv')
    ]),
])

# Callback to update KPI card
@app.callback(
    Output('kpi-card', 'children'),
    [Input('age-group-filter', 'value'),
     Input('encounter-class-filter', 'value')]
)
def update_kpi_card(age_group, encounter_class):
    query = "SELECT COUNT(*) as count FROM patients WHERE readmission_risk >= 0.3909"
    params = []
    if age_group != 'All':
        query += " AND AGE_GROUP = ?"
        params.append(age_group)
    if encounter_class != 'All':
        query += " AND ENCOUNTERCLASS = ?"
        params.append(encounter_class)
    df = pd.read_sql(query, conn, params=params)
    return str(df['count'].iloc[0])

# Callback to update scatter plot
@app.callback(
    Output('scatter-plot', 'figure'),
    [Input('age-group-filter', 'value'),
     Input('encounter-class-filter', 'value')]
)
def update_scatter_plot(age_group, encounter_class):
    query = "SELECT AGE, ENCOUNTERCLASS, readmission_risk FROM patients"
    params = []
    conditions = []
    if age_group != 'All':
        conditions.append("AGE_GROUP = ?")
        params.append(age_group)
    if encounter_class != 'All':
        conditions.append("ENCOUNTERCLASS = ?")
        params.append(encounter_class)
    if conditions:
        query += " WHERE " + " AND ".join(conditions)
    df = pd.read_sql(query, conn, params=params)
    df['Risk_Level'] = np.where(df['readmission_risk'] >= 0.3909, 'High Risk', 'Low Risk')
    return px.scatter(df, x='AGE', y='readmission_risk', color='Risk_Level', 
                      color_discrete_map={'High Risk': 'red', 'Low Risk': 'blue'}, 
                      title='Readmission Risk vs. Age', 
                      labels={'AGE': 'Patient Age', 'readmission_risk': 'Readmission Risk Score'},
                      hover_data=['ENCOUNTERCLASS'])

# Callback to update bar chart
@app.callback(
    Output('bar-chart', 'figure'),
    [Input('age-group-filter', 'value'),
     Input('encounter-class-filter', 'value')]
)
def update_bar_chart(age_group, encounter_class):
    query = "SELECT ENCOUNTERCLASS, readmission_risk FROM patients"
    params = []
    conditions = []
    if age_group != 'All':
        conditions.append("AGE_GROUP = ?")
        params.append(age_group)
    if encounter_class != 'All':
        conditions.append("ENCOUNTERCLASS = ?")
        params.append(encounter_class)
    if conditions:
        query += " WHERE " + " AND ".join(conditions)
    df = pd.read_sql(query, conn, params=params)
    avg_risk_by_encounter = df.groupby('ENCOUNTERCLASS')['readmission_risk'].mean().reset_index()
    avg_risk_by_encounter['Color'] = np.where(avg_risk_by_encounter['readmission_risk'] >= 0.3909, 'red', 'blue')
    fig = go.Figure(data=[go.Bar(x=avg_risk_by_encounter['ENCOUNTERCLASS'], y=avg_risk_by_encounter['readmission_risk'],
                                 marker_color=avg_risk_by_encounter['Color'], text=avg_risk_by_encounter['ENCOUNTERCLASS'],
                                 textposition='auto')])
    fig.update_layout(title='Average Readmission Risk by Encounter Class',
                      xaxis_title='Encounter Class', yaxis_title='Average Risk Score', showlegend=False)
    return fig

# Callback to update pie chart
@app.callback(
    Output('pie-chart', 'figure'),
    [Input('age-group-filter', 'value'),
     Input('encounter-class-filter', 'value')]
)
def update_pie_chart(age_group, encounter_class):
    query = "SELECT AGE_GROUP FROM patients WHERE readmission_risk >= 0.3909"
    params = []
    conditions = []
    if age_group != 'All':
        conditions.append("AGE_GROUP = ?")
        params.append(age_group)
    if encounter_class != 'All':
        conditions.append("ENCOUNTERCLASS = ?")
        params.append(encounter_class)
    if conditions:
        query += " AND " + " AND ".join(conditions)
    df = pd.read_sql(query, conn, params=params)
    age_group_distribution = df['AGE_GROUP'].value_counts().reset_index()
    age_group_distribution.columns = ['AGE_GROUP', 'Count']
    fig = px.pie(age_group_distribution, names='AGE_GROUP', values='Count',
                 title='Distribution of High-Risk Patients by Age Group', color='AGE_GROUP',
                 color_discrete_map={'0-20': '#FF6F61', '21-40': '#6B5B95', '41-60': '#88B04B',
                                     '61-80': '#F7CAC9', '81+': '#92A8D1'})
    fig.update_traces(textinfo='percent+label', pull=[0.1, 0, 0, 0, 0],
                      hovertemplate='%{label}: %{value} patients (%{percent})<extra></extra>')
    return fig

# Callback to update trend line
@app.callback(
    Output('trend-line', 'figure'),
    [Input('age-group-filter', 'value'),
     Input('encounter-class-filter', 'value')]
)
def update_trend_line(age_group, encounter_class):
    query = "SELECT DATE, readmission_risk FROM patients"
    params = []
    conditions = []
    if age_group != 'All':
        conditions.append("AGE_GROUP = ?")
        params.append(age_group)
    if encounter_class != 'All':
        conditions.append("ENCOUNTERCLASS = ?")
        params.append(encounter_class)
    if conditions:
        query += " WHERE " + " AND ".join(conditions)
    df = pd.read_sql(query, conn, params=params)
    df['DATE'] = pd.to_datetime(df['DATE'], errors='coerce')
    df = df.dropna(subset=['DATE'])
    if not df.empty:
        trend_data = df.groupby(df['DATE'].dt.to_period('D').dt.to_timestamp())['readmission_risk'].mean().reset_index()
        fig = px.line(trend_data, x='DATE', y='readmission_risk',
                      title='Average Readmission Risk Over Time',
                      labels={'DATE': 'Date', 'readmission_risk': 'Average Risk Score'})
        fig.add_hline(y=0.3909, line_dash="dash", line_color="red", annotation_text="High Risk Threshold (0.3909)",
                      annotation_position="top right")
    else:
        fig = go.Figure()
        fig.update_layout(title='Average Readmission Risk Over Time', xaxis_title='Date', yaxis_title='Average Risk Score')
    return fig

# Callback to update heatmap
@app.callback(
    Output('heatmap', 'figure'),
    [Input('age-group-filter', 'value'),
     Input('encounter-class-filter', 'value')]
)
def update_heatmap(age_group, encounter_class):
    query = "SELECT AGE_GROUP, ENCOUNTERCLASS, readmission_risk FROM patients"
    params = []
    conditions = []
    if age_group != 'All':
        conditions.append("AGE_GROUP = ?")
        params.append(age_group)
    if encounter_class != 'All':
        conditions.append("ENCOUNTERCLASS = ?")
        params.append(encounter_class)
    if conditions:
        query += " WHERE " + " AND ".join(conditions)
    df = pd.read_sql(query, conn, params=params)
    df = df.dropna(subset=['AGE_GROUP', 'ENCOUNTERCLASS'])
    if not df.empty:
        heatmap_data = df.groupby(['AGE_GROUP', 'ENCOUNTERCLASS'])['readmission_risk'].mean().reset_index()
        heatmap_data = heatmap_data.pivot(index='AGE_GROUP', columns='ENCOUNTERCLASS', values='readmission_risk')
        return px.imshow(heatmap_data, title='Readmission Risk by Age Group and Encounter Class',
                         labels={'color': 'Average Risk Score'}, color_continuous_scale='Reds')
    else:
        return go.Figure().update_layout(title='Readmission Risk by Age Group and Encounter Class')

# Callback to update the patient table
@app.callback(
    Output('patient-table', 'children'),
    [Input('age-group-filter', 'value'),
     Input('encounter-class-filter', 'value'),
     Input('bar-chart', 'clickData'),
     Input('pie-chart', 'clickData')]
)
def update_table(age_group, encounter_class, bar_click, pie_click):
    query = "SELECT AGE, ENCOUNTERCLASS, AGE_GROUP, readmission_risk FROM patients WHERE readmission_risk >= 0.3909"
    params = []
    conditions = []
    if age_group != 'All':
        conditions.append("AGE_GROUP = ?")
        params.append(age_group)
    if encounter_class != 'All':
        conditions.append("ENCOUNTERCLASS = ?")
        params.append(encounter_class)
    if conditions:
        query += " AND " + " AND ".join(conditions)
    df = pd.read_sql(query, conn, params=params)
    if bar_click:
        selected_encounter = bar_click['points'][0]['x']
        df = df[df['ENCOUNTERCLASS'] == selected_encounter]
    if pie_click:
        selected_age_group = pie_click['points'][0]['label']
        df = df[df['AGE_GROUP'] == selected_age_group]
    if df.empty:
        return dash_table.DataTable(data=[], columns=[{'name': 'No data', 'id': 'nodata'}],
                                    style_table={'overflowX': 'auto'}, style_cell={'textAlign': 'left', 'padding': '5px'},
                                    style_header={'backgroundColor': 'lightgrey', 'fontWeight': 'bold'})
    df['Recommendation'] = np.where(df['readmission_risk'] > 0.5, 'Schedule Follow-Up', 'Monitor Closely')
    table_df = df[['AGE', 'ENCOUNTERCLASS', 'AGE_GROUP', 'readmission_risk', 'Recommendation']].head(10)
    style_data_conditional = [
        {'if': {'filter_query': '{readmission_risk} >= 0.3909'}, 'backgroundColor': 'rgba(255, 0, 0, 0.2)', 'color': 'black'}
    ]
    return dash_table.DataTable(data=table_df.to_dict('records'),
                                columns=[{'name': col, 'id': col} for col in table_df.columns],
                                style_table={'overflowX': 'auto'}, style_cell={'textAlign': 'left', 'padding': '5px'},
                                style_header={'backgroundColor': 'lightgrey', 'fontWeight': 'bold'},
                                style_data_conditional=style_data_conditional)

# Callback to handle CSV download
@app.callback(
    Output('download-dataframe-csv', 'data'),
    [Input('download-button', 'n_clicks'),
     Input('age-group-filter', 'value'),
     Input('encounter-class-filter', 'value'),
     Input('bar-chart', 'clickData'),
     Input('pie-chart', 'clickData')],
    prevent_initial_call=True
)
def download_csv(n_clicks, age_group, encounter_class, bar_click, pie_click):
    query = "SELECT AGE, ENCOUNTERCLASS, AGE_GROUP, readmission_risk FROM patients WHERE readmission_risk >= 0.3909"
    params = []
    conditions = []
    if age_group != 'All':
        conditions.append("AGE_GROUP = ?")
        params.append(age_group)
    if encounter_class != 'All':
        conditions.append("ENCOUNTERCLASS = ?")
        params.append(encounter_class)
    if conditions:
        query += " AND " + " AND ".join(conditions)
    df = pd.read_sql(query, conn, params=params)
    if bar_click:
        selected_encounter = bar_click['points'][0]['x']
        df = df[df['ENCOUNTERCLASS'] == selected_encounter]
    if pie_click:
        selected_age_group = pie_click['points'][0]['label']
        df = df[df['AGE_GROUP'] == selected_age_group]
    if df.empty:
        return dcc.send_data_frame(pd.DataFrame(columns=['No data']).to_csv, "high_risk_patients.csv")
    df['Recommendation'] = np.where(df['readmission_risk'] > 0.5, 'Schedule Follow-Up', 'Monitor Closely')
    table_df = df[['AGE', 'ENCOUNTERCLASS', 'AGE_GROUP', 'readmission_risk', 'Recommendation']]
    return dcc.send_data_frame(table_df.to_csv, "high_risk_patients.csv")

# Run the scheduler and app
if __name__ == '__main__':
    print("Starting scheduler and Dash app on localhost:8050...")
    def run_scheduler():
        while True:
            schedule.run_pending()
            time.sleep(1)

    import threading
    scheduler_thread = threading.Thread(target=run_scheduler, daemon=True)
    scheduler_thread.start()

    app.run(port=8050, host='localhost', debug=True, use_reloader=False, threaded=False)
    print("Dash app should be accessible at http://localhost:8050")

Loading initial processed dataset...
Unique ENCOUNTERCLASS values: ['Ambulatory', 'Outpatient', 'Inpatient', 'Telehealth', 'Home', 'Virtual', 'Emergency', 'Other', 'Urgent Care', 'Observation']
Starting scheduler and Dash app on localhost:8050...


Dash app should be accessible at http://localhost:8050
Error fetching new patients: HTTPConnectionPool(host='localhost', port=5000): Max retries exceeded with url: /new_patients (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7d96d623c990>: Failed to establish a new connection: [Errno 111] Connection refused'))
Error fetching new patients: HTTPConnectionPool(host='localhost', port=5000): Max retries exceeded with url: /new_patients (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7d96e5ac6350>: Failed to establish a new connection: [Errno 111] Connection refused'))
Error fetching new patients: HTTPConnectionPool(host='localhost', port=5000): Max retries exceeded with url: /new_patients (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7d96e5050250>: Failed to establish a new connection: [Errno 111] Connection refused'))
Error fetching new patients: HTTPConnectionPool(host='localhost', port=5000): Max

In [1]:
pip install schedule dash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 51.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 11.6 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.0
    Uninstalling Flask-3.1.0:
      Successfully uninstalled Flask-3.1.0
Note: you may need to restart the kernel to use updated packages.


In [1]:
# Install the correct Scikit-learn version for compatibility
!pip install scikit-learn==1.5.0

from flask import Flask, jsonify
import pandas as pd
import numpy as np
from datetime import datetime
import joblib
import os
import time
import threading

app = Flask(__name__)

# Verify dataset paths
print("Checking available datasets in /kaggle/input/...")
print(os.listdir('/kaggle/input/'))

# Load the raw dataset
print("Loading raw dataset...")
start_time = time.time()
data_path = "/kaggle/input/healthcare-data-csv/healthcare_data.csv"
if not os.path.exists(data_path):
    raise FileNotFoundError(f"Raw dataset at '{data_path}' not found. Please check the path.")
data = pd.read_csv(data_path, low_memory=False)  # Address DtypeWarning
print(f"Dataset loading took {time.time() - start_time:.2f} seconds.")
print(f"Dataset loaded with {len(data)} rows and {len(data.columns)} columns.")

# Reduce dataset size for testing
print("Reducing dataset size for testing...")
data = data.head(10000)  # Use only the first 10,000 rows
print(f"Reduced dataset to {len(data)} rows.")

# Preprocess the data
print("Starting preprocessing...")
start_time = time.time()
try:
    data['START'] = pd.to_datetime(data['START'], errors='coerce')
    birth_dates = pd.to_datetime(data['BIRTHDATE'], errors='coerce')
    time_diff = (datetime.now() - birth_dates).dt.total_seconds() / (365.25 * 24 * 60 * 60)
    data['AGE'] = time_diff.astype(float).round(0).astype(int)
except KeyError as e:
    print(f"Error processing data: {e}. Check column names.")
    data['AGE'] = 0

encounter_class_map = {
    'ambulatory': 'Ambulatory', 'outpatient': 'Outpatient', 'inpatient': 'Inpatient',
    'telehealth': 'Telehealth', 'home': 'Home', 'virtual': 'Virtual',
    'emergency': 'Emergency', 'other': 'Other', 'urgentcare': 'Urgent Care',
    'observation': 'Observation'
}
data['ENCOUNTERCLASS'] = data['ENCOUNTERCLASS'].map(encounter_class_map).fillna('Other')
print(f"Preprocessing took {time.time() - start_time:.2f} seconds.")

# Verify model dataset paths
print("Checking model dataset in /kaggle/input/...")
model_dataset = [d for d in os.listdir('/kaggle/input/') if 'xgboost_model' in d.lower()]
if not model_dataset:
    raise FileNotFoundError("Model dataset not found.")
model_dataset_path = f"/kaggle/input/{model_dataset[0]}"
print(f"Model dataset found: {os.listdir(model_dataset_path)}")

# Dynamically determine the correct subpath
subpath = 'scikitlearn'
if os.path.exists(f"{model_dataset_path}/{subpath}"):
    subpath_files = os.listdir(f"{model_dataset_path}/{subpath}")
    if 'default' in subpath_files:
        subpath += '/default'
    if '1' in os.listdir(f"{model_dataset_path}/{subpath}"):
        subpath += '/1'
    prep_path = f"{model_dataset_path}/{subpath}/preprocessor.pkl"
    model_path = f"{model_dataset_path}/{subpath}/xgboost_model.pkl"
else:
    prep_path = f"{model_dataset_path}/preprocessor.pkl"
    model_path = f"{model_dataset_path}/xgboost_model.pkl"

if not os.path.exists(prep_path):
    raise FileNotFoundError(f"Preprocessor file at '{prep_path}' not found.")
if not os.path.exists(model_path):
    raise FileNotFoundError(f"Model file at '{model_path}' not found.")
preprocessor = joblib.load(prep_path)
model = joblib.load(model_path)
print("Preprocessor and model loaded successfully.")

# Define columns used in training
numerical_cols = [
    'BASE_ENCOUNTER_COST', 'TOTAL_CLAIM_COST', 'PAYER_COVERAGE', 'LAT', 'LON',
    'HEALTHCARE_EXPENSES', 'HEALTHCARE_COVERAGE', 'INCOME', 'AGE'
]
categorical_cols = ['ENCOUNTERCLASS', 'MARITAL', 'RACE', 'ETHNICITY']

total_patients = len(data)
current_index = 0

@app.route('/new_patients', methods=['GET'])
def get_new_patients():
    global current_index
    batch_size = 10
    end_index = min(current_index + batch_size, total_patients)
    if end_index <= current_index:
        return jsonify({"error": "No more patients to serve"}), 400
    batch = data.iloc[current_index:end_index].copy()

    try:
        for col in numerical_cols + categorical_cols:
            if col not in batch.columns:
                batch[col] = np.nan
        for col in categorical_cols:
            batch[col] = batch[col].astype(str)
        for col in numerical_cols:
            batch[col] = pd.to_numeric(batch[col], errors='coerce')
        batch[numerical_cols] = batch[numerical_cols].fillna(batch[numerical_cols].mean())
        for col in categorical_cols:
            batch[col] = batch[col].fillna(batch[col].mode()[0])
        batch_transformed = preprocessor.transform(batch)
        risk_scores = model.predict_proba(batch_transformed)[:, 1]
        batch['readmission_risk'] = risk_scores
        batch['DATE'] = datetime.now().isoformat()
        new_patients = batch[['AGE', 'ENCOUNTERCLASS', 'readmission_risk', 'DATE']].to_dict(orient='records')
    except Exception as e:
        return jsonify({"error": f"Prediction failed: {e}"}), 500

    current_index = end_index
    if current_index >= total_patients:
        current_index = 0
    return jsonify(new_patients)

if __name__ == '__main__':
    print("Starting Flask app on port 5001 without reloader...")
    def run_flask():
        try:
            print("Binding to port 5001...")
            app.run(host='0.0.0.0', port=5001, debug=False, use_reloader=False)
            print("Flask app is running on http://0.0.0.0:5001")
        except Exception as e:
            print(f"Failed to start Flask app: {e}")

    # Run Flask in a background thread
    flask_thread = threading.Thread(target=run_flask, daemon=True)
    flask_thread.start()
    print("Flask app running in background. Use http://127.0.0.1:5001/new_patients to test.")
    # Keep the notebook interactive
    time.sleep(5)  # Wait briefly to ensure Flask starts
    print("Testing Flask server...")
    try:
        import requests
        response = requests.get('http://127.0.0.1:5001/new_patients', timeout=10)
        print("Flask server responded successfully:")
        print(response.json())
    except Exception as e:
        print(f"Failed to connect to Flask server: {e}")

Checking available datasets in /kaggle/input/...
['healthcare-data-engineered-csv-gz', 'healthcare-data-timeseries-csv-gz', 'xgboost_model.pkl', 'healthcare-step4-processed1', 'healthcare-data-csv', 'healthcare-data-final-csv-gz']
Loading raw dataset...
Dataset loading took 20.84 seconds.
Dataset loaded with 1000000 rows and 43 columns.
Reducing dataset size for testing...
Reduced dataset to 10000 rows.
Starting preprocessing...
Preprocessing took 0.02 seconds.
Checking model dataset in /kaggle/input/...
Model dataset found: ['scikitlearn']
Preprocessor and model loaded successfully.
Starting Flask app on port 5001 without reloader...
Binding to port 5001...
 * Serving Flask app '__main__'
Flask app running in background. Use http://127.0.0.1:5001/new_patients to test.
 * Debug mode: off
Testing Flask server...
Flask server responded successfully:
[{'AGE': 23, 'DATE': '2025-05-19T09:56:00.819557', 'ENCOUNTERCLASS': 'Outpatient', 'readmission_risk': 0.058884064356486}, {'AGE': 23, 'DATE

In [2]:
import requests
response = requests.get('http://127.0.0.1:5001/new_patients')
print(response.status_code)
print(response.json())

200
[{'AGE': 65, 'DATE': '2025-05-19T09:56:05.574447', 'ENCOUNTERCLASS': 'Ambulatory', 'readmission_risk': 0.09532517691453297}, {'AGE': 65, 'DATE': '2025-05-19T09:56:05.574447', 'ENCOUNTERCLASS': 'Ambulatory', 'readmission_risk': 0.09375143051147461}, {'AGE': 65, 'DATE': '2025-05-19T09:56:05.574447', 'ENCOUNTERCLASS': 'Emergency', 'readmission_risk': 0.12761741876602173}, {'AGE': 65, 'DATE': '2025-05-19T09:56:05.574447', 'ENCOUNTERCLASS': 'Ambulatory', 'readmission_risk': 0.10419641683499019}, {'AGE': 65, 'DATE': '2025-05-19T09:56:05.574447', 'ENCOUNTERCLASS': 'Ambulatory', 'readmission_risk': 0.09380381802717845}, {'AGE': 65, 'DATE': '2025-05-19T09:56:05.574447', 'ENCOUNTERCLASS': 'Ambulatory', 'readmission_risk': 0.10419641683499019}, {'AGE': 57, 'DATE': '2025-05-19T09:56:05.574447', 'ENCOUNTERCLASS': 'Ambulatory', 'readmission_risk': 0.06235965465505918}, {'AGE': 57, 'DATE': '2025-05-19T09:56:05.574447', 'ENCOUNTERCLASS': 'Ambulatory', 'readmission_risk': 0.07005156079928081}, {'AG

In [8]:
!lsof -i :5001

COMMAND PID USER   FD   TYPE DEVICE SIZE/OFF NODE NAME
python3  31 root   58u  IPv4  37480      0t0  TCP *:5001 (LISTEN)


In [ ]:
!kill -9 31